## Example workbook for DIC <-> EBSD linking

In [1]:
# Load third-party packages
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt

# Load quat, ebsd and hrdic packages from defdap package
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic

### Load in and display DIC map

In [2]:
#Load in DIC map
DicFilePath = "example_data/"
DicMap = hrdic.Map(DicFilePath, "B00005.txt")

#Set crop
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

Loaded DaVis 8.1.5 data (dimensions: 586 x 510 pixels, sub-window size: 12 x 12 pixels)


In [3]:
DicMap.setScale(micrometrePerPixel=0.02)

In [4]:
# Display max shear map
DicMap.plotMaxShear(plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.1)

### Print stats table

In [5]:
DicMap.printStatsTable(percentiles=['Min', 5, 'Mean', 95,'Max'], components = ['mss', 'f11', 'f22'])

DicMap (dimensions: 543 x 465 pixels, sub-window size: 12 x 12 pixels, number of points: 252495)

Component	Min	P5	Mean	P95	Max	
mss		0.01	0.39	1.95	4.39	12.74	
f11		-9.26	-0.34	2.13	5.15	16.30	
f22		-13.78	-3.77	-1.10	0.88	7.63	



### Effective shear strain histogram

In [6]:
histData = DicMap.crop(DicMap.max_shear).flatten()

fig, ax = plt.subplots(1)

counts, bins, bars = ax.hist(histData, bins=np.linspace(0,0.1,100), color='r', histtype='step')

ax.set_xlabel("Effective shear strain")
ax.set_ylabel("Frequency")

fig.show()

### Load in EBSD map then calculate local misorientation and display it

In [7]:
# Load in EBSD map
EbsdFilePath = "example_data/Map Data 2-DIC area"
EbsdMap = ebsd.Map(EbsdFilePath, "cubic")

# Load in slip system definitions (optional)
EbsdMap.loadSlipSystems("cubic_FCC")

# Rotate the map 180 degrees if necessary
EbsdMap.transformData()

# Create the internal array of quaternions for orientation calculations
EbsdMap.buildQuatArray()

# Find boundaries with given misorientation tolerance (in degrees)
EbsdMap.findBoundaries(boundDef = 6)

# Find grains, minGrainSize is the minimum number of pixels for a grain
EbsdMap.findGrains(minGrainSize = 10)

# Calculate grain local misorientation (Grain Reference Orientation Deviation (GROD) w.r.t grain mean orientations)
EbsdMap.calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 1006 x 996 pixels, step size: 0.1 um)
Done                                               

In [8]:
EbsdMap.plotEulerMap()

In [9]:
EbsdMap.plotMisOriMap(plotGBs=True, vmin=0, vmax=4)

### Plot IPF of all grains mean orientation

In [10]:
# Calculate grain mean orientations (but is already done here as part of calcGrainMisOri above)
# EbsdMap.calcGrainAvOris()

grainMeanOris = [grain.refOri for grain in EbsdMap]
refDir = np.array([1, 0, 0])

Quat.plotIPF(grainMeanOris, refDir, EbsdMap.crystalSym)

### Find a grain of interest (click around the map)

In [11]:
EbsdMap.locateGrainID()

Grain ID: 302
Grain ID: 334
Grain ID: 123


### Plot misorientation and IPF for isolated grain

In [12]:
currEbsdGrain = EbsdMap[EbsdMap.currGrainId]

In [13]:
currEbsdGrain.plotMisOri()

In [14]:
currEbsdGrain.plotMisOri(component=3)

In [15]:
plot = currEbsdGrain.plotOriSpread(direction=refDir, c='b', s=1, alpha=0.2)
currEbsdGrain.plotRefOri(direction=refDir, c='k', s=100, plot=plot)

### Plot phase map

In [16]:
EbsdMap.plotPhaseMap()

## Link EBSD map to DIC map

### Define homologous points

In [18]:
# Example of use of interactive tool coming soon...

In [17]:
# Set homologous points   (x, y)
DicMap.homogPoints = np.array((
    (303, 408), (120, 392), (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776), (236, 790), (466, 160)
))

### Display map with homologous points

In [18]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [19]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the EBSD and DIC map

In [20]:
DicMap.linkEbsdMap(EbsdMap)

### Plot max shear with boundaries

In [21]:
DicMap.plotMaxShear(plotGBs=True, plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.1)

### Detect grains in the DIC map

In [22]:
DicMap.findGrains(minGrainSize=10)

Done                                               

### Locate a grain of interest (click around)

In [25]:
DicMap.locateGrainID(displaySelected=True)
# displaySelected will show the selcted grain in a separate figure window

Grain ID: 154
Number of bands detected: 1
Grain ID: 53
Number of bands detected: 3
Grain ID: 155
Number of bands detected: 1


### Plot max shear for isolated grain

In [26]:
currDicGrain = DicMap[DicMap.currGrainId]

In [27]:
currDicGrain.plotMaxShear()

### Plot a histogram of it

In [28]:
plt.figure()

plt.hist(currDicGrain.maxShearList, bins=100, histtype='step');

plt.xlabel("Effective shear strain")
plt.ylabel("Frequency")

Text(0, 0.5, 'Frequency')

### Plot misorientation for isolated grain

In [29]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
currEbsdGrain = EbsdMap[ebsdGrainId]

In [30]:
currEbsdGrain.plotMisOri()

### Plot max shear for isolated grain with slip traces

In [31]:
currDicGrain.calcSlipTraces()
currDicGrain.plotMaxShear(plotSlipTraces=True)